In [2]:
import cryptocompare
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta
plt.rcParams["figure.figsize"] = (18,7)

In [3]:
# how much data you want
limit_data = 200 #(hours)
# What you wanna trade
coin = 'SUSHI'
# Base currency 
currency = 'BTC'

In [4]:
data = cryptocompare.get_historical_price_hour(coin, curr=currency, limit= limit_data, exchange='Binance')

In [14]:
# mean = []
time = []
high = []
low = []
open1 = []
close1 = []
volume = []
for i in range(len(data)):
    hi = data[i]['high']
    lo = data[i]['low']
    op = data[i]['open']
    cl = data[i]['close']
    vl = data[i]['volumefrom']
#     mean.append((hi+lo)/2)
    time.append(i)
    high.append(hi)
    low.append(lo)
    open1.append(op)
    close1.append(cl)
    volume.append(vl)
#df = pd.DataFrame(list(zip(pd.Series(pd.date_range('2000', freq='H', periods=limit_data)),high,low,open1,close1,volume,mean)), columns =['Date','high','low','open','close','volumefrom', 'mean']) 

time_delta = timedelta(hours=limit_data, minutes=0, seconds=0, microseconds=0) # minus hours 
date_at_start = datetime.now() - time_delta
days = pd.date_range(date_at_start, date_at_start + timedelta(limit_data/24), freq='H') # if freq is in hours then divide limit_data by 24

df = pd.DataFrame({'Date': days,'Open':open1, 'High': high, 'Low' : low, 'Close': close1, 'Volume' : volume})
df = df.set_index('Date')

In [15]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2020-12-17 00:51:37.559191,0.000136,0.000140,0.000136,0.000139,163708.36
2020-12-17 01:51:37.559191,0.000139,0.000140,0.000137,0.000138,120690.50
2020-12-17 02:51:37.559191,0.000138,0.000139,0.000133,0.000133,406213.80
2020-12-17 03:51:37.559191,0.000133,0.000135,0.000132,0.000133,335096.27
2020-12-17 04:51:37.559191,0.000133,0.000134,0.000131,0.000133,262231.87
...,...,...,...,...,...
2020-12-25 04:51:37.559191,0.000109,0.000110,0.000106,0.000109,192442.05
2020-12-25 05:51:37.559191,0.000109,0.000110,0.000108,0.000108,75679.92
2020-12-25 06:51:37.559191,0.000108,0.000114,0.000108,0.000111,187431.53


In [28]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA

class SmaCross(Strategy):
    def init(self):
        price_close = self.data.Close
        self.ma1 = self.I(SMA, price_close, 7)    #7 and 39 gives 54% return SUSHI hourly chart
        self.ma2 = self.I(SMA, price_close, 39)

    def next(self):
        self.data.Volume
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()




bt = Backtest(df, SmaCross, commission=.005, exclusive_orders=True)
stats = bt.run()
print(stats)
bt.plot()

Start                     2020-12-17 00:51...
End                       2020-12-25 08:51...
Duration                      8 days 08:00:00
Exposure Time [%]                     75.1244
Equity Final [$]                      10273.4
Equity Peak [$]                       11196.5
Return [%]                            2.73367
Buy & Hold Return [%]                -20.2878
Return (Ann.) [%]                     198.551
Volatility (Ann.) [%]                 243.038
Sharpe Ratio                         0.816954
Sortino Ratio                           5.265
Calmar Ratio                           11.242
Max. Drawdown [%]                    -17.6615
Avg. Drawdown [%]                    -5.52423
Max. Drawdown Duration        2 days 19:00:00
Avg. Drawdown Duration        1 days 01:00:00
# Trades                                    5
Win Rate [%]                               60
Best Trade [%]                        6.69197
Worst Trade [%]                      -7.89355
Avg. Trade [%]                    

<ipython-input-28-594be0b4e50a>:25: UserWarning: 'Can't superimpose OHLC data with rule 'None'(index datetime resolution: 'microsecond'). Skipping.
  bt.plot()
